# Malware Detection in Network Traffic Data

## Imports and spark setup

In [1]:
import pyspark
from tqdm import tqdm
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SQLContext, SparkSession

spark = SparkSession.builder.appName('Malware').config("spark.executor.memory", "8g").getOrCreate()

#sc = spark.sparkContext
#sqlsc = SQLContext(spark)

import os
os.environ['HADOOP_HOME'] = 'C:/dummy/hadoop_home'

In [2]:
print(spark.sparkContext.getConf().getAll())

[('spark.executor.id', 'driver'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'), ('spark.driver.port', '54529'), ('spark.app.name', 'Malware'), ('spark.rdd.compress', 'True'), ('spark.driver.host', '192.1

## Presentation

In [3]:
data = r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv"

In [4]:
df = spark.read.option("escape","\"").option("delimiter", "|").csv(data, header='true', inferSchema='true')

In [5]:
rows, cols = df.count(), len(df.columns)
rows, cols

(1008748, 23)

In [6]:
df.printSchema()

root
 |-- ts: double (nullable = true)
 |-- uid: string (nullable = true)
 |-- id.orig_h: string (nullable = true)
 |-- id.orig_p: double (nullable = true)
 |-- id.resp_h: string (nullable = true)
 |-- id.resp_p: double (nullable = true)
 |-- proto: string (nullable = true)
 |-- service: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- orig_bytes: string (nullable = true)
 |-- resp_bytes: string (nullable = true)
 |-- conn_state: string (nullable = true)
 |-- local_orig: string (nullable = true)
 |-- local_resp: string (nullable = true)
 |-- missed_bytes: double (nullable = true)
 |-- history: string (nullable = true)
 |-- orig_pkts: double (nullable = true)
 |-- orig_ip_bytes: double (nullable = true)
 |-- resp_pkts: double (nullable = true)
 |-- resp_ip_bytes: double (nullable = true)
 |-- tunnel_parents: string (nullable = true)
 |-- label: string (nullable = true)
 |-- detailed-label: string (nullable = true)



In [7]:
df.show(5)

+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|                 ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|  51524.0| 65.127.233.163|     23.0|  tcp|      -|2.999051|         0|         0|     

## Dataset concatenation / data assesment / schema corrections [preprocessing]

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

# Define your custom schema
custom_schema = StructType([
    StructField("ts", DoubleType(), True),
    StructField("uid", StringType(), True),
    StructField("id.orig_h", StringType(), True),
    StructField("id.orig_p", StringType(), True),
    StructField("id.resp_h", StringType(), True),
    StructField("id.resp_p", StringType(), True),
    StructField("proto", StringType(), True),
    StructField("service", StringType(), True),
    StructField("duration", DoubleType(), True),
    StructField("orig_bytes", DoubleType(), True),
    StructField("resp_bytes", DoubleType(), True),
    StructField("conn_state", StringType(), True),
    StructField("local_orig", StringType(), True),
    StructField("local_resp", StringType(), True),
    StructField("missed_bytes", DoubleType(), True),
    StructField("history", StringType(), True),
    StructField("orig_pkts", DoubleType(), True),
    StructField("orig_ip_bytes", DoubleType(), True),
    StructField("resp_pkts", DoubleType(), True),
    StructField("resp_ip_bytes", DoubleType(), True),
    StructField("tunnel_parents", StringType(), True),
    StructField("label", StringType(), True),
    StructField("detailed-label", StringType(), True),
])

# List of file paths
file_paths = [
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-1-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-3-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-9-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-20-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-21-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-34-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-35-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-42-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-44-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-48-1conn.log.labeled.csv",
    r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-60-1conn.log.labeled.csv",
]

# Initialize an empty DataFrame with the custom schema
df = spark.createDataFrame(spark.sparkContext.emptyRDD(), custom_schema)

# Use tqdm for progress bar
for file_path in tqdm(file_paths, desc="Reading files", unit="file"):
    df_temp = spark.read.option("escape", "\"").option("delimiter", "|").csv(file_path, header=True, schema=custom_schema)
    df = df.union(df_temp)

# Show the first 5 rows and print the schema
df.show(5)
df.printSchema()


Reading files: 100%|██████████| 11/11 [00:00<00:00, 46.27file/s]


+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|                 ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|    51524| 65.127.233.163|       23|  tcp|      -|2.999051|       0.0|       0.0|     

In [9]:
#replacing dots with underscores in column names to avoid errors
df = df.toDF(*(c.replace('.', '_') for c in df.columns))

In [10]:
#looking for the coluns that have the '-' value to later work on them
from pyspark.sql.functions import col

columns_with_dash = [col_name for col_name in df.columns if df.filter(col(col_name) == "-").count() > 0]
print("Columns with '-' values:", columns_with_dash)

Columns with '-' values: ['service', 'local_orig', 'local_resp', 'history', 'tunnel_parents', 'detailed-label']


In [11]:
from pyspark.sql.functions import col, when
from pyspark.sql import SparkSession

def replace_dash_with_nan(df, columns):

    result_df = df
    for column in columns:
        result_df = result_df.withColumn(column, when(col(column) == '-', None).otherwise(col(column)))

    return result_df

df = replace_dash_with_nan(df, columns_with_dash)

In [12]:
columns_with_dash = [col_name for col_name in df.columns if df.filter(col(col_name) == "-").count() > 0]
print("Columns with '-' values:", columns_with_dash)

Columns with '-' values: []


In [13]:
from pyspark.sql.functions import isnan, when, count, col, isnull
missing = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

+---+---+---------+---------+---------+---------+-----+--------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
| ts|uid|id_orig_h|id_orig_p|id_resp_h|id_resp_p|proto| service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|label|detailed-label|
+---+---+---------+---------+---------+---------+-----+--------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|  0|  0|        0|        0|        0|        0|    0|24982603|15265888|  15265888|  15265888|         0|  25000600|  25000600|           0|  25116|        0|            0|        0|            0|      25000600|    0|      17951931|
+---+---+---------+---------+---------+---------+-----+--------+

In [14]:
from pyspark.sql.functions import from_unixtime, col

df = df.withColumn("formatted_ts", from_unixtime("ts").cast("timestamp"))
df.show(5)

+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+-------------------+
|                 ts|               uid|      id_orig_h|id_orig_p|      id_resp_h|id_resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|       formatted_ts|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+-------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|    51524| 65.127.233.163|

## Data understanding

### dimension and content analysis

In [15]:
rows, cols = df.count(), len(df.columns)
print(f'Dimension of the Dataframe is: {(rows,cols)}')

Dimension of the Dataframe is: (25000600, 24)


In [16]:
num_cols = [item[0] for item in df.dtypes if item[1] != 'string']
print('Le colonne numeriche sono {}'.format(len(num_cols)))
print(num_cols)

Le colonne numeriche sono 10
['ts', 'duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'formatted_ts']


In [17]:
non_num_cols = [item[0] for item in df.dtypes if item[1] == 'string']
print('Le colonne non numeriche sono {}'.format(len(non_num_cols)))
print(non_num_cols)

Le colonne non numeriche sono 14
['uid', 'id_orig_h', 'id_orig_p', 'id_resp_h', 'id_resp_p', 'proto', 'service', 'conn_state', 'local_orig', 'local_resp', 'history', 'tunnel_parents', 'label', 'detailed-label']


### Check for missing labels for each file to understandand labelling consistency

In [18]:
from pyspark.sql.functions import isnan, when, count, col, isnull


def check_nan(spark, column_to_check="", label_only=False):
    # List of file numbers
    file_numbers = [1, 3, 9, 20, 21, 34, 35, 42, 44, 48, 60]
    
    for number in tqdm(file_numbers, desc="Processing files", unit="file"):
    
        file_path = r"C:\Users\Vincenzo\Projects\DDAM_data\malware\CTU-IoT-Malware-Capture-{}-1conn.log.labeled.csv".format(number)
        df = spark.read.option("escape", "\"").option("delimiter", "|").csv(file_path, header='true', inferSchema='true')
        df = df.toDF(*(c.replace('.', '_') for c in df.columns))

        print("", flush=True)
        # Check for missing values
        if label_only:
            missing = df.select([count(when(isnull("detailed-label"), "detailed-label")).alias("missing_count")])
            print("Missing values in file {}: ".format(number))
            missing.select("missing_count").show()

        elif column_to_check:
            missing = df.select([count(when(isnull(column_to_check), column_to_check)).alias("missing_count")])
            print("Missing values in file {}: ".format(number))
            missing.select("missing_count").show()

        elif column_to_check == "all":
            missing = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])
            print("Missing values in file {}: ".format(number))
            missing.show()

        else:
            print("Please enter a valid column name or enter True to check the label column only")

# Check for missing values in the label column only
check_nan(spark, label_only=True)


Processing files:   0%|          | 0/11 [00:00<?, ?file/s]


Missing values in file 1: 


Processing files:   9%|▉         | 1/11 [00:01<00:17,  1.78s/file]

+-------------+
|missing_count|
+-------------+
|            0|
+-------------+




Processing files:  18%|█▊        | 2/11 [00:02<00:09,  1.09s/file]

...

Missing values in file 44: 
+-------------+
|missing_count|
+-------------+
|           15|
+-------------+


Missing values in file 48: 


Processing files:  91%|█████████ | 10/11 [00:32<00:03,  3.42s/file]

+-------------+
|missing_count|
+-------------+
|      3388871|
+-------------+


Missing values in file 60: 


Processing files: 100%|██████████| 11/11 [00:36<00:00,  3.35s/file]

+-------------+
|missing_count|
+-------------+
|      3578457|
+-------------+



### Check for distinct values and raw statistics

In [19]:
from pyspark.sql.functions import col, countDistinct

non_num_df = df.select(non_num_cols)

def count_distinct_values(df):
    result = {}
    columns = df.columns

    # Use tqdm to create a progress bar
    for column in tqdm(columns, desc="Counting Distinct Values", unit="column"):
        distinct_count = df.select(column).agg(countDistinct(column)).collect()[0][0]
        result[column] = distinct_count

    return result

distinct_counts = count_distinct_values(non_num_df)

for column, count in distinct_counts.items():
    print(f"Column '{column}' has {count} distinct values.")

Counting Distinct Values:   0%|          | 0/14 [00:00<?, ?column/s]

Counting Distinct Values: 100%|██████████| 14/14 [06:35<00:00, 28.27s/column]

Column 'uid' has 25000600 distinct values.
Column 'id_orig_h' has 21442 distinct values.
Column 'id_orig_p' has 102304 distinct values.
Column 'id_resp_h' has 11654577 distinct values.
Column 'id_resp_p' has 75872 distinct values.
Column 'proto' has 3 distinct values.
Column 'service' has 6 distinct values.
Column 'conn_state' has 13 distinct values.
Column 'local_orig' has 0 distinct values.
Column 'local_resp' has 0 distinct values.
Column 'history' has 263 distinct values.
Column 'tunnel_parents' has 0 distinct values.
Column 'label' has 7 distinct values.
Column 'detailed-label' has 6 distinct values.


In [20]:
# Use the describe function to get statistical summary

summary = df.describe()
summary.show()


+-------+--------------------+------------------+------------+------------------+-----------+------------------+--------+-------+-----------------+--------------------+--------------------+----------+----------+----------+-----------------+--------+------------------+-----------------+--------------------+------------------+--------------+--------------------+--------------+
|summary|                  ts|               uid|   id_orig_h|         id_orig_p|  id_resp_h|         id_resp_p|   proto|service|         duration|          orig_bytes|          resp_bytes|conn_state|local_orig|local_resp|     missed_bytes| history|         orig_pkts|    orig_ip_bytes|           resp_pkts|     resp_ip_bytes|tunnel_parents|               label|detailed-label|
+-------+--------------------+------------------+------------+------------------+-----------+------------------+--------+-------+-----------------+--------------------+--------------------+----------+----------+----------+-----------------+----

In [21]:
# num_summary = summary.select(*num_cols)
num_summary = summary.select(*(["summary"] + num_cols))
# non_num_summary = summary.select(*non_num_cols)
non_num_summary = summary.select(*(["summary"] + non_num_cols))

print("Summary of numeric columns:")
num_summary.show()

print("Summary of non-numeric columns:")
non_num_summary.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `formatted_ts` cannot be resolved. Did you mean one of the following? [`orig_pkts`, `duration`, `label`, `missed_bytes`, `orig_bytes`].;
'Project [summary#22825, ts#22459, duration#22467, orig_bytes#22468, resp_bytes#22469, missed_bytes#22473, orig_pkts#22475, orig_ip_bytes#22476, resp_pkts#22477, resp_ip_bytes#22478, 'formatted_ts]
+- Project [summary#22825, element_at(ts#22348, summary#22825, None, false) AS ts#22459, element_at(uid#22353, summary#22825, None, false) AS uid#22460, element_at(id_orig_h#22358, summary#22825, None, false) AS id_orig_h#22461, element_at(id_orig_p#22363, summary#22825, None, false) AS id_orig_p#22462, element_at(id_resp_h#22368, summary#22825, None, false) AS id_resp_h#22463, element_at(id_resp_p#22373, summary#22825, None, false) AS id_resp_p#22464, element_at(proto#22378, summary#22825, None, false) AS proto#22465, element_at(service#22383, summary#22825, Non